# Explore files containing ICU admission information

In [6]:
import pandas as pd

pd.set_option('display.max_rows', None)

# Explore mapping file

In [ ]:
import os
import pandas as pd

from pathlib import Path


dataDf = pd.read_csv(os.environ['EHR_DATA_BASE'] + '/blood_pos_cohort_20240531/data/data_matrix.csv')
mappingDf = pd.read_csv(Path(os.environ['EHR_DATA_BASE'], 'saur_cohort_20250113', 'source', 'patient_tube_id_mapping_full.tsv'), sep='\t')

dataDf.shape, mappingDf.shape

((116754, 194), (3117, 18))

In [2]:
studyMappingDf = mappingDf[['PATIENT_ID', 'EPISODE_ID', 'location_additional_02']].merge(
    dataDf[['person_id', 'visit_occurrence_id']].drop_duplicates(),
    how='inner',
    left_on=['PATIENT_ID', 'EPISODE_ID'],
    right_on=['person_id', 'visit_occurrence_id']
).drop(
    columns=['person_id', 'visit_occurrence_id']
).drop_duplicates()
studyMappingDf

,PATIENT_ID,EPISODE_ID,location_additional_02
0,2268214.0,12918522.0,A-7EA;HAEM
1,50056.0,13266809.0,A-7EA;Bone Marrow & Transplant
2,62567.0,12648702.0,A-HITH;Urology
3,138108.0,14737781.0,Ward AC1
4,211314.0,13955099.0,Ward 7 East;nan
...,...,...,...
795,2677962.0,17459522.0,Ward 7 East
796,2684042.0,17566160.0,Emergency Dept
797,2207334.0,15581117.0,Ward 7 West
801,2638484.0,16969999.0,Ward AC2


In [3]:
studyMappingDf.location_additional_02.value_counts().reset_index()

,location_additional_02,count
0,Ward 7 East,118
1,Ward 7 East;nan,56
2,nan;nan,52
3,A-7EA;HAEM,36
4,ICU,33
...,...,...
142,Better at home (D) Disability,1
143,Ward 2F,1
144,Better at Home (A),1
145,Ward 2 Discharge Zone,1


In [4]:
studyMappingDf[studyMappingDf.location_additional_02.notna() & studyMappingDf.location_additional_02.str.lower().str.contains('icu')].location_additional_02.value_counts().reset_index()

,location_additional_02,count
0,ICU,33
1,ICU;nan,12
2,ICU;Cardiology - Heart Failure,6
3,ICU;Lung Transplanatation Service (RES1),3
4,ICU;Cardio Thoracic,3
5,ICU;HAEM,3
6,ICU;Bone Marrow & Transplant,3
7,ICU;Lung Transplantation,2
8,ICU;AHPB & General Surgery,2
9,ICU;Infectious Diseases,1


In [5]:
studyMappingDf[['PATIENT_ID', 'EPISODE_ID']].drop_duplicates().shape[0]

654

In [6]:
studyMappingDf[studyMappingDf.location_additional_02.notna() & studyMappingDf.location_additional_02.str.lower().str.contains('icu')][['PATIENT_ID', 'EPISODE_ID']].drop_duplicates().shape[0]

70

In [7]:
round(studyMappingDf[studyMappingDf.location_additional_02.notna() & studyMappingDf.location_additional_02.str.lower().str.contains('icu')][['PATIENT_ID', 'EPISODE_ID']].drop_duplicates().shape[0]/studyMappingDf[['PATIENT_ID', 'EPISODE_ID']].drop_duplicates().shape[0]*100, 2)

10.7

In [8]:
studyMappingDf['ICU_ADMISSION'] = studyMappingDf.location_additional_02.notna() & studyMappingDf.location_additional_02.str.lower().str.contains('icu')
studyMappingDf

,PATIENT_ID,EPISODE_ID,location_additional_02,ICU_ADMISSION
0,2268214.0,12918522.0,A-7EA;HAEM,False
1,50056.0,13266809.0,A-7EA;Bone Marrow & Transplant,False
2,62567.0,12648702.0,A-HITH;Urology,False
3,138108.0,14737781.0,Ward AC1,False
4,211314.0,13955099.0,Ward 7 East;nan,False
...,...,...,...,...
795,2677962.0,17459522.0,Ward 7 East,False
796,2684042.0,17566160.0,Emergency Dept,False
797,2207334.0,15581117.0,Ward 7 West,False
801,2638484.0,16969999.0,Ward AC2,False


In [9]:
studyMappingDf[['PATIENT_ID', 'EPISODE_ID', 'ICU_ADMISSION']].drop_duplicates().ICU_ADMISSION.value_counts()/studyMappingDf[['PATIENT_ID', 'EPISODE_ID', 'ICU_ADMISSION']].drop_duplicates().shape[0]*100

ICU_ADMISSION
False    89.441931
True     10.558069
Name: count, dtype: float64

## Explore master journey file

### `SuperBugJourney` file

In [10]:
import os
import pandas as pd


from pathlib import Path


superBugJourneyDf = pd.read_csv(Path(os.environ['EHR_DATA_BASE'], 'blood_pos_cohort_20240531', 'etc', 'SuperBugJourney-72hr-2024-03-27.csv'))
superBugJourneyDf

/tmp/ipykernel_854676/4105351322.py:8: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  superBugJourneyDf = pd.read_csv(Path(os.environ['EHR_DATA_BASE'], 'blood_pos_cohort_20240531', 'etc', 'SuperBugJourney-72hr-2024-03-27.csv'))


,PATIENT_ID,JOURNEY_ID,TOTAL_EPISODES,JOURNEY_START_DATETIME,JOURNEY_END_DATETIME,BED_DAYS,HITH_DAYS,BAH_DAYS,CONTRACT_DAYS,ED_HOURS,ICU_HOURS,CAMPUS_TRANSFERS,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15
0,2171902,380495,1,1/18/2017 16:05,4/1/2019 12:45,803,0,0,0,0.0,0.00,0,NaN,NaN,NaN,NaN
1,2191482,496223,1,12/14/2017 17:18,7/26/2018 13:40,166,117,0,0,0.0,119.63,0,NaN,NaN,NaN,NaN
2,2127789,530095,1,3/26/2018 13:06,7/27/2018 12:00,123,121,0,0,0.0,0.00,0,NaN,NaN,NaN,NaN
3,1001078,530312,2,3/27/2018 6:30,9/27/2018 15:00,184,0,0,0,0.0,1776.73,1,NaN,NaN,NaN,NaN
4,2153229,536357,1,4/13/2018 16:56,8/9/2018 10:37,80,55,0,0,0.0,0.00,0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46253,2084529,18787436,2,3/22/2024 15:54,NaN,1,0,0,0,3.6,0.00,0,NaN,NaN,NaN,NaN
46254,2139095,18787607,1,3/22/2024 16:00,3/22/2024 19:25,1,0,0,0,0.0,0.00,0,NaN,NaN,NaN,NaN
46255,2094045,18787910,1,3/22/2024 19:48,NaN,0,0,0,0,2.9,0.00,0,NaN,NaN,NaN,NaN
46256,2024172,18788021,1,3/22/2024 21:46,NaN,0,0,0,0,0.9,0.00,0,NaN,NaN,NaN,NaN


In [14]:
superBugJourneyDf['ICU_ADMISSION'] = (superBugJourneyDf.ICU_HOURS > 0)
superBugJourneyDf.ICU_ADMISSION.value_counts()

ICU_ADMISSION
False    45533
True       725
Name: count, dtype: int64

In [21]:
icuadmissionsDf = superBugJourneyDf.groupby(by=['PATIENT_ID', 'JOURNEY_ID']).agg({'ICU_ADMISSION': 'any'}).reset_index()
icuadmissionsDf

,PATIENT_ID,JOURNEY_ID,ICU_ADMISSION
0,18849,12790706,False
1,18849,13476453,False
2,18849,13477689,False
3,18849,13569580,False
4,18849,13572789,False
...,...,...,...
46253,2680034,17495361,False
46254,2680486,17502337,False
46255,2681377,17518169,False
46256,2682433,17536805,False


In [22]:
icuadmissionsDf.ICU_ADMISSION.value_counts()

ICU_ADMISSION
False    45533
True       725
Name: count, dtype: int64

In [28]:
mergedDf = dataDf.merge(
    icuadmissionsDf,
    how='left',
    left_on=['person_id', 'visit_occurrence_id'],
    right_on=['PATIENT_ID', 'JOURNEY_ID']
).drop(
    columns=['PATIENT_ID', 'JOURNEY_ID']
).fillna(False)
mergedDf

/tmp/ipykernel_854676/579351500.py:8: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ).fillna(False)


,person_id,visit_occurrence_id,measurement_date,admittime_adm,dischtime_adm,vitals_Peripheral oxygen saturation_avg,vitals_Respiratory rate_avg,vitals_Temperature_avg,vitals_Systolic blood pressure_avg,vitals_Diastolic blood pressure_avg,...,labs_Total protein measurement_last,labs_Globulin_last,labs_Calcium_last,labs_Phosphate measurement_last,labs_Magnesium measurement_last,deathtime_adm,death_7_day,death_14_day,death_30_day,ICU_ADMISSION
0,2152832,113016,30/12/2014,03/11/2015,30/12/2014,95.000000,20.064682,36.100000,140.000000,101.000000,...,63.00,36.0,2.2600,1.0400,0.7800,False,0,0,0,False
1,2152832,133225,31/03/2015,03/11/2015,31/03/2015,98.000000,18.710677,36.200000,135.000000,102.000000,...,57.00,31.0,2.2200,1.8200,0.9400,False,0,0,0,False
2,1548773,145028,03/06/2015,20/09/2015,03/06/2015,98.000000,20.000000,36.666095,108.000000,65.000000,...,56.00,24.0,2.3300,1.2500,0.7200,False,0,0,0,False
3,2081586,178895,22/10/2015,01/10/2016,22/10/2015,99.000000,16.954422,36.460042,126.000000,78.000000,...,64.00,27.0,2.4100,0.9700,0.6300,False,0,0,0,False
4,2152832,189928,08/12/2015,03/11/2015,08/12/2015,98.000000,17.643095,36.500000,91.000000,57.000000,...,55.00,33.0,2.1500,2.0500,0.6100,False,0,0,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116749,2449953,18035869,21/08/2023,11/07/2023,17/08/2023,97.833333,16.500000,36.471429,125.666667,73.500000,...,62.00,39.0,2.2500,0.9000,0.7500,False,0,0,0,False
116750,2449953,18035869,22/08/2023,11/07/2023,17/08/2023,99.000000,19.000000,36.250000,112.750000,76.750000,...,60.00,37.0,2.2400,0.9900,0.7500,False,0,0,0,False
116751,852009,18036769,18/08/2023,18/11/2022,18/08/2023,100.000000,16.000000,36.000000,113.333333,55.333333,...,63.00,18.0,2.3500,1.3600,0.7400,False,0,0,0,False
116752,2105227,18039086,17/08/2023,29/11/2022,18/08/2023,95.666667,16.833333,36.275000,127.400000,75.400000,...,74.22,38.0,2.3535,1.0059,0.7884,False,0,0,0,False


In [30]:
mergedDf[['person_id', 'visit_occurrence_id', 'ICU_ADMISSION']].drop_duplicates().ICU_ADMISSION.value_counts()/mergedDf[['person_id', 'visit_occurrence_id', 'ICU_ADMISSION']].drop_duplicates().shape[0]*100

ICU_ADMISSION
False    98.027962
True      1.972038
Name: count, dtype: float64

### `SuperBugSuperEpisode` file

In [ ]:
import os
import pandas as pd

from pathlib import Path


superBugJourneyEpisodeDf = pd.read_csv(Path(os.environ['EHR_DATA_BASE'], 'blood_pos_cohort_20240531', 'etc', 'SuperBugSuperEpisode-72hr-2024-03-27.csv'))
superBugJourneyEpisodeDf

/tmp/ipykernel_854676/1476996103.py:8: DtypeWarning: Columns (10,17,26,31,37,44) have mixed types. Specify dtype option on import or set low_memory=False.
  superBugJourneyEpisodeDf = pd.read_csv(Path(os.environ['EHR_DATA_BASE'], 'blood_pos_cohort_20240531', 'etc', 'SuperBugSuperEpisode-72hr-2024-03-27.csv'))


,PATIENT_ID,JOURNEY_ID,JOURNEY_SEQUENCE,PARENT_EPISODE_ID,EPISODE_ID,IDENTIFIER,EPISODE_TYPE,EPISODE_SEQUENCE,FACILITY,FACILITY_CODE,...,ALL_CARE_TYPES,TOTAL_ED_HOURS,TOTAL_BED_DAYS,TOTAL_HITH_DAYS,TOTAL_BAH_DAYS,TOTAL_CONTRACT_DAYS,ENCOUNTER_DISCHARGE_METHOD,ENCOUNTER_DISCHARGE_TO,ROW_TYPE,Column1
0,1001529,4381012,1,4381012,584898,I5282484,Inpatient,2,The Alfred,1010,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Episodes
1,1006096,4449933,1,4449933,582927,I5280479,Inpatient,2,The Alfred,1010,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Episodes
2,1006096,4449934,1,4449934,593561,I5291389,Inpatient,2,The Alfred,1010,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Episodes
3,1012027,4487128,1,4487128,607654,I5306038,Inpatient,2,The Alfred,1010,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Episodes
4,1019178,4584278,1,4584278,602854,I5301040,Inpatient,2,The Alfred,1010,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Episodes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95486,2221044,17376468,174,18685425,18685425,I6059549,Inpatient,1,Sandringham Hospital,1360,...,SSNR,4,NaN,1.0,0.0,0.0,0.0,Home/Private residence/Accommodation,NaN,Summary
95487,2221044,17376468,175,18723473,18723473,I6064438,Inpatient,1,Sandringham Hospital,1360,...,SSNR,4,NaN,1.0,0.0,0.0,0.0,Home/Private residence/Accommodation,NaN,Summary
95488,2221044,17376468,176,18723474,18723474,I6064439,Inpatient,1,Sandringham Hospital,1360,...,SSNR,4,NaN,1.0,0.0,0.0,0.0,Home/Private residence/Accommodation,NaN,Summary
95489,2130904,13400080,74,13917585,13917585,I5555173,Inpatient,1,Sandringham Hospital,1360,...,MSED,4,NaN,1.0,0.0,0.0,0.0,Home/Private residence/Accommodation,NaN,Summary


In [49]:
superBugJourneyEpisodeDf.columns

Index(['PATIENT_ID', 'JOURNEY_ID', 'JOURNEY_SEQUENCE', 'PARENT_EPISODE_ID',
       'EPISODE_ID', 'IDENTIFIER', 'EPISODE_TYPE', 'EPISODE_SEQUENCE',
       'FACILITY', 'FACILITY_CODE', 'TRANSFERRED_FROM',
       'EPISODE_START_DATETIME', 'EPISODE_END_DATETIME', 'BED_DAYS',
       'HITH_DAYS', 'BAH_DAYS', 'CARE_TYPE', 'CARE_TYPE_CODE', 'ADMIT_UNIT',
       'ADMIT_WARD', 'VIRTUAL_ED_FLAG', 'ICU_HOURS', 'DISCHARGE_UNIT',
       'DISCHARGE_WARD', 'DISCHARGE_METHOD', 'DISCHARGE_TO',
       'FINAL_EPISODE_ID', 'FINAL_IDENTIFIER', 'FINAL_EPISODE_TYPE',
       'FINAL_EPISODE_CARE_TYPE', 'FINAL_EPISODE_CARE_TYPE_CODE',
       'NUMBER_OF_EPISODES', 'ENCOUNTER_START_DATETIME',
       'ENCOUNTER_END_DATETIME', 'FINAL_WARD', 'FINAL_UNIT', 'ALL_CARE_TYPES',
       'TOTAL_ED_HOURS', 'TOTAL_BED_DAYS', 'TOTAL_HITH_DAYS', 'TOTAL_BAH_DAYS',
       'TOTAL_CONTRACT_DAYS', 'ENCOUNTER_DISCHARGE_METHOD',
       'ENCOUNTER_DISCHARGE_TO', 'ROW_TYPE', 'Column1'],
      dtype='object')

In [50]:
superBugJourneyEpisodeDf[superBugJourneyEpisodeDf.ADMIT_WARD.str.lower().str.contains('icu', na=False)].ADMIT_WARD.value_counts()

ADMIT_WARD
ICU    944
Name: count, dtype: int64

## Explore ICU admission labels (current)

In [8]:
import os
import pandas as pd


from pathlib import Path


labelsDf = pd.read_csv(Path(os.environ['EHR_DATA_BASE'], 'blood_pos_cohort_20240119', 'final', 'icu_admission_labels.csv'))
labelsDf

,person_id,visit_occurrence_id,ICU_ADMISSION
0,2152832,113016,False
1,2152832,133225,False
2,1548773,145028,False
3,2081586,178895,False
4,2152832,189928,False
...,...,...,...
23169,2196440,18035002,False
23170,2449953,18035869,False
23171,852009,18036769,False
23172,2105227,18039086,False


In [9]:
studyLabelsDf = labelsDf.merge(
    dataDf[['person_id', 'visit_occurrence_id']],
    how='inner',
    on=['person_id', 'visit_occurrence_id']
).drop_duplicates()
studyLabelsDf

,person_id,visit_occurrence_id,ICU_ADMISSION
0,2152832,113016,False
1,2152832,133225,False
2,1548773,145028,False
3,2081586,178895,False
4,2152832,189928,False
...,...,...,...
116744,2196440,18035002,False
116745,2449953,18035869,False
116751,852009,18036769,False
116752,2105227,18039086,False


In [10]:
studyLabelsDf[['person_id', 'visit_occurrence_id']].drop_duplicates().shape[0]

23174

In [11]:
studyLabelsDf[studyLabelsDf.ICU_ADMISSION][['person_id', 'visit_occurrence_id']].drop_duplicates().shape[0]

364

In [12]:
round(studyLabelsDf[studyLabelsDf.ICU_ADMISSION][['person_id', 'visit_occurrence_id']].drop_duplicates().shape[0]/studyLabelsDf[['person_id', 'visit_occurrence_id']].drop_duplicates().shape[0]*100, 2)

1.57

## Explore ICU admissions labels

In [13]:
import os
import pandas as pd


from pathlib import Path


icuAdmissionsDf = pd.read_csv(Path(os.environ['EHR_DATA_BASE'], 'blood_pos_cohort_20240119', 'additional_data', '2024-blood-ICU-admissions.csv'))
icuAdmissionsDf

,ICU_ADMISSION_ID,PARENT_EPISODE_ID,PATIENT_ID,EPISODE_ID,REFERRAL_DATETIME,REFERRAL_ACTION_DATETIME,REFERRAL_ORDER_ID,ADMIT_DATETIME,ADMISSION_SOURCE,HOSPITAL_ADMISSION_SOURCE,...,LAST_UPDATED_BY_PROVIDER_INT_ID,LAST_UPDATE_DATETIME,RECORD_STATUS,ORIGINATING_SYSTEM,ORIGINATING_SYSTEM_PK,CREATED_BY,MODIFIED_BY,CREATED_DATETIME,MODIFIED_DATETIME,SUPPRESSED
0,8,338686,2121884,338686,NaN,NaN,NaN,2016-11-01 00:45:00.000,OT / RECOVERY,Home,...,752991,2017-02-01 16:56:00,A,Cerner,11442,LINKEN,LINKEN,2023-09-18 16:28:57.820,2024-02-07 10:37:57.327,0
1,24,354273,316516,354273,NaN,NaN,NaN,2016-11-03 00:05:00.000,OT / RECOVERY,Home,...,743892,2017-07-14 14:10:54,A,Cerner,11458,SYSTEM,SYSTEM,2023-09-18 16:28:57.820,2024-02-07 10:37:57.327,0
2,33,350242,2099069,350242,NaN,NaN,NaN,2016-11-04 01:53:00.000,WARD,Home,...,752991,2017-02-01 12:10:22,A,Cerner,11467,LINKEN,LINKEN,2023-09-18 16:28:57.820,2024-02-07 10:37:57.327,0
3,43,355223,1001248,355223,NaN,NaN,NaN,2016-11-05 12:33:00.000,OT / RECOVERY,Home,...,752991,2017-02-01 16:36:42,A,Cerner,11477,LINKEN,LINKEN,2023-09-18 16:28:57.820,2024-02-07 10:37:57.327,0
4,67,332295,652290,332295,NaN,NaN,NaN,2016-11-09 10:13:00.000,WARD,Other Hospital - ICU,...,752991,2017-02-02 15:52:55,A,Cerner,11501,LINKEN,LINKEN,2023-09-18 16:28:57.820,2024-02-07 10:37:57.327,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3806,29319,18567602,2131388,18567602,NaN,NaN,NaN,2024-01-23 22:42:00.000,OT / RECOVERY,Home,...,830378,2024-02-15 14:36:32,A,Cerner,37790,SYSTEM,MALLERL,2024-01-23 22:48:51.250,2024-02-15 14:56:33.550,0
3807,29366,18580467,2162592,18580839,2024-01-28 14:13:44.000,2024-01-28 14:14:21.000,96069325.0,2024-01-28 19:49:00.000,EMERGENCY,Home,...,830378,2024-02-14 08:00:30,A,Cerner,37818,SYSTEM,MALLERL,2024-01-28 19:59:08.687,2024-02-14 08:16:53.040,0
3808,29382,18584202,2196022,18586220,2024-01-29 19:16:53.000,2024-01-29 19:18:12.000,96111538.0,2024-01-30 02:13:00.000,EMERGENCY,Other Acute Hospital (not ICU/ED),...,830378,2024-02-06 11:36:56,A,Cerner,37837,SYSTEM,MALLERL,2024-01-30 04:24:48.603,2024-02-07 10:37:57.327,0
3809,29441,18606263,2045525,18606593,NaN,NaN,NaN,2024-02-04 06:15:00.000,WARD,Home,...,832714,2024-02-15 15:52:58,A,Cerner,37900,SYSTEM,COLLINSKAT,2024-02-04 06:50:10.170,2024-02-15 16:15:41.600,0


In [14]:
icuAdmissionsDf.shape

(3811, 63)

In [19]:
icuAdmissionsDf.merge(
    labelsDf[labelsDf.ICU_ADMISSION][['person_id', 'visit_occurrence_id']],
    how='inner',
    left_on=['PATIENT_ID', 'EPISODE_ID'],
    right_on=['person_id', 'visit_occurrence_id']
).shape

(497, 65)

In [83]:
studyIcuAdmissionsDf = icuAdmissionsDf[['PATIENT_ID', 'EPISODE_ID', 'ADMIT_DATETIME']].drop_duplicates().merge(
    dataDf[['person_id', 'visit_occurrence_id', 'admittime_adm']].drop_duplicates(),
    how='inner',
    left_on=['PATIENT_ID', 'EPISODE_ID'],
    right_on=['person_id', 'visit_occurrence_id']
).rename(
    columns={'ADMIT_DATETIME': 'ICU_ADMIT_DATETIME'}
).drop(
    columns=['PATIENT_ID', 'EPISODE_ID']
)
studyIcuAdmissionsDf.ICU_ADMIT_DATETIME = pd.to_datetime(studyIcuAdmissionsDf.ICU_ADMIT_DATETIME.str.slice(start=0, stop=10))
studyIcuAdmissionsDf.admittime_adm = pd.to_datetime(studyIcuAdmissionsDf.admittime_adm, format='%d/%m/%Y')
studyIcuAdmissionsDf['ICU_ADMISSION'] = (studyIcuAdmissionsDf.admittime_adm < studyIcuAdmissionsDf.ICU_ADMIT_DATETIME) & (studyIcuAdmissionsDf.admittime_adm > (studyIcuAdmissionsDf.ICU_ADMIT_DATETIME - pd.Timedelta(days=14)))
studyIcuAdmissionsDf = studyIcuAdmissionsDf.drop(columns=['admittime_adm', 'ICU_ADMIT_DATETIME']).drop_duplicates()
# studyIcuAdmissionsDf.groupby([])
studyIcuAdmissionsDf

,person_id,visit_occurrence_id,ICU_ADMISSION
0,316516,354273,False
1,1001248,355223,False
2,307501,355359,False
3,2007372,328741,False
5,2090434,510183,False
...,...,...,...
706,2402469,17260371,False
707,418202,17279019,True
708,2155638,17332590,False
709,2543700,17249022,False


In [84]:
studyIcuAdmissionsDf.ICU_ADMISSION.value_counts()

ICU_ADMISSION
False    514
True      81
Name: count, dtype: int64

In [87]:
mergedDataDf = dataDf.merge(
    studyIcuAdmissionsDf[studyIcuAdmissionsDf.ICU_ADMISSION],
    how='left',
    on=['person_id', 'visit_occurrence_id']
).drop_duplicates()
mergedDataDf.ICU_ADMISSION = mergedDataDf.ICU_ADMISSION.fillna(False)
mergedDataDf

/tmp/ipykernel_849509/3322135411.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  mergedDataDf.ICU_ADMISSION = mergedDataDf.ICU_ADMISSION.fillna(False)


,person_id,visit_occurrence_id,measurement_date,admittime_adm,dischtime_adm,vitals_Peripheral oxygen saturation_avg,vitals_Respiratory rate_avg,vitals_Temperature_avg,vitals_Systolic blood pressure_avg,vitals_Diastolic blood pressure_avg,...,labs_Total protein measurement_last,labs_Globulin_last,labs_Calcium_last,labs_Phosphate measurement_last,labs_Magnesium measurement_last,deathtime_adm,death_7_day,death_14_day,death_30_day,ICU_ADMISSION
0,2152832,113016,30/12/2014,03/11/2015,30/12/2014,95.000000,20.064682,36.100000,140.000000,101.000000,...,63.00,36.0,2.2600,1.0400,0.7800,NaN,0,0,0,False
1,2152832,133225,31/03/2015,03/11/2015,31/03/2015,98.000000,18.710677,36.200000,135.000000,102.000000,...,57.00,31.0,2.2200,1.8200,0.9400,NaN,0,0,0,False
2,1548773,145028,03/06/2015,20/09/2015,03/06/2015,98.000000,20.000000,36.666095,108.000000,65.000000,...,56.00,24.0,2.3300,1.2500,0.7200,NaN,0,0,0,False
3,2081586,178895,22/10/2015,01/10/2016,22/10/2015,99.000000,16.954422,36.460042,126.000000,78.000000,...,64.00,27.0,2.4100,0.9700,0.6300,NaN,0,0,0,False
4,2152832,189928,08/12/2015,03/11/2015,08/12/2015,98.000000,17.643095,36.500000,91.000000,57.000000,...,55.00,33.0,2.1500,2.0500,0.6100,NaN,0,0,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116749,2449953,18035869,21/08/2023,11/07/2023,17/08/2023,97.833333,16.500000,36.471429,125.666667,73.500000,...,62.00,39.0,2.2500,0.9000,0.7500,NaN,0,0,0,False
116750,2449953,18035869,22/08/2023,11/07/2023,17/08/2023,99.000000,19.000000,36.250000,112.750000,76.750000,...,60.00,37.0,2.2400,0.9900,0.7500,NaN,0,0,0,False
116751,852009,18036769,18/08/2023,18/11/2022,18/08/2023,100.000000,16.000000,36.000000,113.333333,55.333333,...,63.00,18.0,2.3500,1.3600,0.7400,NaN,0,0,0,False
116752,2105227,18039086,17/08/2023,29/11/2022,18/08/2023,95.666667,16.833333,36.275000,127.400000,75.400000,...,74.22,38.0,2.3535,1.0059,0.7884,NaN,0,0,0,False


In [92]:
mergedDataDf[['person_id', 'visit_occurrence_id', 'ICU_ADMISSION']].drop_duplicates().ICU_ADMISSION.value_counts()/mergedDataDf[['person_id', 'visit_occurrence_id', 'ICU_ADMISSION']].drop_duplicates().shape[0]*100

ICU_ADMISSION
False    99.65047
True      0.34953
Name: count, dtype: float64

## Explore transfers file

In [12]:
import os
import pandas as pd


from pathlib import Path


dataDf = pd.read_csv(os.environ['EHR_DATA_BASE'] + '/blood_pos_cohort_20240531/data/data_matrix.csv')
transferDf = pd.read_csv(Path(os.environ['EHR_DATA_BASE'], 'blood_pos_cohort_20240119', 'additional_data', 'IMP4_Bed_transfer_20220121_updated.csv'))

dataDf.shape, transferDf.shape

((116754, 194), (9166, 18))

In [9]:
transferDf.TO_WARD_CODE.value_counts().reset_index()[:10]

,TO_WARD_CODE,count
0,Discharged,3971
1,ICU,457
2,5EA,448
3,7EA,383
4,ADU,349
5,6WS,254
6,3EA,240
7,3WS,215
8,AOR,183
9,4WA,179


In [35]:
studyTransfersDf = transferDf[['PATIENT_ID', 'TO_WARD_CODE', 'TRANSFER_OUT_WARD_BED_DATE']].drop_duplicates().merge(
    dataDf[['person_id', 'visit_occurrence_id', 'dischtime_adm']].drop_duplicates(),
    how='inner',
    left_on=['PATIENT_ID'],
    right_on=['person_id']
).drop_duplicates().drop(
    columns=['PATIENT_ID']
)
# studyTransfersDf.TRANSFER_OUT_WARD_BED_DATE = pd.to_datetime(studyTransfersDf.TRANSFER_OUT_WARD_BED_DATE.str.slice(start=0, stop=10))
# studyTransfersDf.dischtime_adm = pd.to_datetime(studyTransfersDf.dischtime_adm, format='%d/%m/%Y')
studyTransfersDf['ICU_ADMISSION'] = studyTransfersDf.TO_WARD_CODE.str.lower().str.contains('icu')
studyTransfersDf = studyTransfersDf.drop(columns=['TO_WARD_CODE', 'TRANSFER_OUT_WARD_BED_DATE', 'dischtime_adm']).drop_duplicates()
# studyIcuAdmissionsDf.groupby([])
# studyTransfersDf.shape
studyTransfersDf[:2]

,person_id,visit_occurrence_id,ICU_ADMISSION
0,2208001,497095,False
1,2208001,511786,False


In [37]:
studyTransfersDf.ICU_ADMISSION.value_counts()/studyTransfersDf.shape[0]*100

ICU_ADMISSION
False    57.514881
True     42.485119
Name: count, dtype: float64